In [1]:
import sys
import struct
import os
from scapy.all import *
from matplotlib import pyplot as plt
from prettytable import PrettyTable
from utils import *

filename = "zoom/zoom_noaudio.pcapng"
a = rdpcap(filename)
sessions = a.sessions()

In [15]:
Connections = []
my_ip = "172.16.105.110"
    

for session in sessions:
    for id_p,packet in enumerate(sessions[session]):
        if(packet.haslayer(ARP) or packet.haslayer(DNS)):
            continue
             
        if(packet.haslayer(IP)):
            if(packet.haslayer(TCP)):                
                flag = False
                idx = len(Connections)
                for i,con in enumerate(Connections):
                    if match_TCP(con, packet):
                        flag = True
                        idx = i
                        break
                if not flag:
                    if packet[IP].src==my_ip:
                        Connections.append({
                            's_ip':packet[IP].src,'d_ip':packet[IP].dst,
                            's_port':packet[TCP].sport,'d_port':packet[TCP].dport,
                            'type':"TCP",
                            'snd_packets':[],
                            'rcv_packets':[]
                        })
                    else:
                        Connections.append({
                            's_ip':packet[IP].dst,'d_ip':packet[IP].src,
                            's_port':packet[TCP].dport,'d_port':packet[TCP].sport,
                            'type':"TCP",
                            'snd_packets':[],
                            'rcv_packets':[]
                        })
                    
                if packet[IP].src==my_ip:
                    Connections[idx]['snd_packets'].append({
                        "seq":packet[TCP].seq, "len":len(packet[TCP].payload),
                        "flag":packet[TCP].flags, "order": id_p
                    })
                else:
                    Connections[idx]['rcv_packets'].append({
                        "seq":packet[TCP].seq, "len":len(packet[TCP].payload),
                        "flag":packet[TCP].flags, "order": id_p
                    })
                pass
        
            elif(packet.haslayer(UDP)):
                flag = False
                idx = len(Connections)
                for i,con in enumerate(Connections):
                    if match_UDP(con, packet):
                        flag = True
                        idx=i
                        break
                if not flag:
                    if packet[IP].src==my_ip:
                        Connections.append({
                            's_ip':packet[IP].src,'d_ip':packet[IP].dst,
                            's_port':packet[UDP].sport,'d_port':packet[UDP].dport,
                            'type':"UDP",
                            'snd_packets':[],
                            'rcv_packets':[]
                        })
                    else:
                        Connections.append({
                            's_ip':packet[IP].dst,'d_ip':packet[IP].src,
                            's_port':packet[UDP].dport,'d_port':packet[UDP].sport,
                            'type':"UDP",
                            'snd_packets':[],
                            'rcv_packets':[]
                        })
                    
                if packet[IP].src==my_ip:
                    Connections[idx]['snd_packets'].append({
                        "len":len(packet[UDP].payload),
                            "order": id_p
                    })
                        
                else:
                    Connections[idx]['rcv_packets'].append({
                        "len":len(packet[UDP].payload),
                        "order": id_p
                    })
                pass
            else:
                pass
#                 break

            

In [16]:
x = PrettyTable()

my_ip = "172.16.105.110"
x.field_names = ["Connection", "Type", "Num Pkt H2R", "Num Pkt R2H", "Len Data H2R", "Len Data R2H"]

for con in Connections:
    x.add_row([f"{con['s_ip']}:{con['s_port']}->{con['d_ip']}:{con['d_port']}", 
               con['type'], 
               len(con['snd_packets']), len(con['rcv_packets']), 
               sum([pkt['len'] for pkt in con['snd_packets']]), 
               sum([pkt['len'] for pkt in con['rcv_packets']])])
    

print(x)

+-------------------------------------------+------+-------------+-------------+--------------+--------------+
|                 Connection                | Type | Num Pkt H2R | Num Pkt R2H | Len Data H2R | Len Data R2H |
+-------------------------------------------+------+-------------+-------------+--------------+--------------+
|  172.16.105.110:60452->170.114.10.89:443  | TCP  |      2      |      3      |      0       |      43      |
|  172.16.105.110:60454->170.114.10.89:443  | TCP  |      2      |      2      |      0       |      37      |
|  172.16.105.110:42166->198.251.131.69:443 | TCP  |      8      |      10     |     703      |     230      |
| 172.16.105.110:48778->198.251.131.69:8801 | UDP  |      4      |      4      |     324      |     324      |
|  172.16.105.110:56786->170.114.10.83:443  | TCP  |      2      |      3      |      0       |      43      |
| 172.16.105.110:41758->198.251.131.69:8801 | UDP  |      4      |      4      |     324      |     324      |
|

In [17]:
Connections

[{'s_ip': '172.16.105.110',
  'd_ip': '170.114.10.89',
  's_port': 60452,
  'd_port': 443,
  'type': 'TCP',
  'snd_packets': [{'seq': 2968727396,
    'len': 0,
    'flag': <Flag 16 (A)>,
    'order': 1},
   {'seq': 2968727396, 'len': 0, 'flag': <Flag 16 (A)>, 'order': 4}],
  'rcv_packets': [{'seq': 136651007,
    'len': 31,
    'flag': <Flag 24 (PA)>,
    'order': 0},
   {'seq': 136651038, 'len': 6, 'flag': <Flag 17 (FA)>, 'order': 2},
   {'seq': 136651038, 'len': 6, 'flag': <Flag 17 (FA)>, 'order': 3}]},
 {'s_ip': '172.16.105.110',
  'd_ip': '170.114.10.89',
  's_port': 60454,
  'd_port': 443,
  'type': 'TCP',
  'snd_packets': [{'seq': 970789286,
    'len': 0,
    'flag': <Flag 16 (A)>,
    'order': 6},
   {'seq': 970789286, 'len': 0, 'flag': <Flag 16 (A)>, 'order': 8}],
  'rcv_packets': [{'seq': 4291186869,
    'len': 31,
    'flag': <Flag 24 (PA)>,
    'order': 5},
   {'seq': 4291186900, 'len': 6, 'flag': <Flag 17 (FA)>, 'order': 7}]},
 {'s_ip': '172.16.105.110',
  'd_ip': '198.251.